In [37]:
import pandas as pd
import numpy as np
import math
from operator import itemgetter

traindf=pd.read_csv('train.csv')
testdf=pd.read_csv('test.csv')

#From Previous Assignment: Label Encoding, where strings match the label, assign an integer and append to an array
def labelTest(data):
    petal_labels=[]
    for i in data:
        if i == 'Iris-setosa':
            petal_labels.append(0)
        if i == 'Iris-versicolor':
            petal_labels.append(1)
        if i == 'Iris-virginica':
            petal_labels.append(2)
    petal_labels = np.asarray(petal_labels)
    return petal_labels
petal_labels=labelTest(testdf["Class"])




In [42]:

#finds square of each column entry of ith row between traindf and testdf. 
def euclidianDistance(data1, data2, length):
    distance = 0
    for i in range(length):
        distance += np.square(data1[i] - data2[i])
    distance = np.sqrt(distance)
    return distance

#Outer loop runs for a total of 15 test entries and inner loop runs for 135 times for train entries
def SortedDistances(traindf, testdf):
    UnSorted_distances = []
    for i in range(len(testdf)):
        test_row = testdf.iloc[i]
        test_distances = []
        
        
        for j in range(len(traindf)):
            train_row = traindf.iloc[j]
            test_distances.append((euclidianDistance(test_row, train_row, 4), train_row[4]))
            
        UnSorted_distances.append(test_distances)

#the above funtion appends the distance found in a tuple with the label of the data being used. It makes an array of arrays 
#with unsorted tuples
#the function below will sort the tuples within the array of arrays
    sortDistances = []
    for i in range(len(UnSorted_distances)):
        sortDistances.append(sorted(UnSorted_distances[i],key=itemgetter(0)))
    return sortDistances

#find nearest K neighbors using mode values for labels
def findNeighbors(sortDistances, k):
    Neighborlabels = []
    for i in range(len(sortDistances)):
        nearest_k = []
        for j in range(k):
            nearest_k.append(sortDistances[i][j][1])
        Neighborlabels.append(max(set(nearest_k), key=nearest_k.count))
    return Neighborlabels






In [43]:
def KNN(train, test, k):
    sortDistances = SortedDistances(traindf, testdf)
    labels = findNeighbors(sortDistances, 5)
    predicts = labelTest(labels)
    return predicts





In [ ]:
k1 = KNN(traindf, testdf,1)
k3 = KNN(traindf, testdf,3)
k5 = KNN(traindf, testdf,5)

In [44]:
def evaluateFinal(prediction, expected):
    
    truep = [0,0,0]
    truen = [0,0,0]
    falsep = [0,0,0]
    faslen = [0,0,0]
    confusion_matrix = np.full((3,3), 0)

    for E, P in zip(expected, prediction): 
        confusion_matrix[P][E] = confusion_matrix[P][E] + 1
        
    
    truep[0] = confusion_matrix[0][0]
    falsep[0] = (confusion_matrix[:, 0]).sum() - truep[0]
    faslen[0] = (confusion_matrix[0, :]).sum() - truep[0]
    truen[0] = (confusion_matrix.sum()) - truep[0] - falsep[0] - faslen[0]
    
    truep[1] = confusion_matrix[1][1]
    falsep[1] = (confusion_matrix[:, 1]).sum() - truep[1]
    faslen[1] = (confusion_matrix[1, :]).sum() - truep[1]
    truen[1] = (confusion_matrix.sum()) - truep[1] - falsep[1] - faslen[1]
        
    truep[2] = confusion_matrix[2][2]
    falsep[2] = (confusion_matrix[:, 2]).sum() - truep[2]
    faslen[2] = (confusion_matrix[2, :]).sum() - truep[2]
    truen[2] = (confusion_matrix.sum()) - truep[2] - falsep[2] - faslen[2]
    print("This is the confusion matrix: \n")
    print(confusion_matrix)
    microA_precision = (sum(truep)/(sum(falsep)+sum(truep)))
    microA_recall = (sum(truep)/(sum(faslen)+sum(truep)))
    microA_accuracy = ((sum(truep)+sum(truen))/(sum(faslen)+sum(truep)+sum(falsep)+sum(truen)))
    microA_f1score = (microA_precision * microA_recall) / (microA_precision + microA_recall)
    microA_f1score = microA_f1score * 2
    
    macroA_precision = []
    macroA_recall = []
    macroA_accuracy = []
    
    macroA_precision.append(truep[0] / (truep[0] + falsep[0]))
    macroA_recall.append(truep[0] / (truep[0] + faslen[0]))
    macroA_accuracy.append((truep[0] + truen[0]) / (truep[0] + truen[0] + falsep[0] + faslen[0]))
    
    macroA_precision.append(truep[1] / (truep[1] + falsep[1]))
    macroA_recall.append(truep[1] / (truep[1] + faslen[1]))
    macroA_accuracy.append((truep[1] + truen[1]) / (truep[1] + truen[1] + falsep[1] + faslen[1]))
    
    macroA_precision.append(truep[2] / (truep[2] + falsep[2]))
    macroA_recall.append(truep[2] / (truep[2] + faslen[2]))
    macroA_accuracy.append((truep[2] + truen[2]) / (truep[2] + truen[2] + falsep[2] + faslen[2]))

    sum_macroA_precision = (sum(macroA_precision))/3
    sum_macroA_recall = (sum(macroA_recall))/3
    sum_macroA_accuracy = (sum(macroA_accuracy))/3

    macroA_f1score = (sum_macroA_precision * sum_macroA_recall) / (sum_macroA_precision + sum_macroA_recall)
    macroA_f1score  = 2 * macroA_f1score 
    
    print("\nMicro Average Precision: ", microA_precision)
    print("\nMicro Average Recall: ", microA_recall)
    print("\nMicro Average Accuracy: ", microA_accuracy)
    print("\nMicro Average F1-Score: ", microA_f1score)
    print("\n")
    print("\nMacro Average Precision: ", sum_macroA_precision)
    print("\nMacro Average Recall: ", sum_macroA_recall)
    print("\nMacro Average Accuracy: ", sum_macroA_accuracy)
    print("\nMacro Average F1-Score: ", macroA_f1score)
    


print("For K = 1")
evaluateFinal(k1, petal_labels) 
print("\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
print("\n\nFor K = 3")
evaluateFinal(k3, petal_labels)  
print("\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
print("\n\nFor K = 5")
evaluateFinal(k5, petal_labels)  

For K = 1
This is the confusion matrix: 

[[5 0 0]
 [0 5 0]
 [0 0 5]]

Micro Average Precision:  1.0

Micro Average Recall:  1.0

Micro Average Accuracy:  1.0

Micro Average F1-Score:  1.0



Macro Average Precision:  1.0

Macro Average Recall:  1.0

Macro Average Accuracy:  1.0

Macro Average F1-Score:  1.0

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


For K = 3
This is the confusion matrix: 

[[5 0 0]
 [0 5 0]
 [0 0 5]]

Micro Average Precision:  1.0

Micro Average Recall:  1.0

Micro Average Accuracy:  1.0

Micro Average F1-Score:  1.0



Macro Average Precision:  1.0

Macro Average Recall:  1.0

Macro Average Accuracy:  1.0

Macro Average F1-Score:  1.0

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


For K = 5
This is the confusion matrix: 

[[5 0 0]
 [0 5 0]
 [0 0 5]]

Micro Average Precision:  1.0

Micro Average Recall:  1.0

Micro Average Accuracy:  1.0

Micro Average F1-Score:  1.0



Macro Average Precision:  1.0

Macro Average Recall:  1.0
